---
title: "Monte Carlo Simulation: Comparing Linear, Ridge, and Lasso Regression"
format: md
jupyter: python3
---


## Objective

This Monte Carlo simulation compares the performance of three regression models:
- Linear Regression
- Ridge Regression
- Lasso Regression

We will simulate 30 datasets in total:
- 10 datasets with no correlation between predictors $x_1$ and $x_2$
- 10 datasets with mild correlation
- 10 datasets with high correlation

The models will be evaluated on Mean Squared Error (MSE). The results will be summarized using the mean and standard deviation of the MSE for each combination of model and correlation structure.

## Setup


In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

np.random.seed(116)

## Data Generation Function


In [ ]:
def generate_data(n=100, correlation=0.0):
    mean = [0, 0]
    cov = [[1, correlation], [correlation, 1]]
    X = np.random.multivariate_normal(mean, cov, size=n)
    x1, x2 = X[:, 0], X[:, 1]
    noise = np.random.normal(0, 1, size=n)
    y = 0 * x1 + 4 * x2 + noise
    return pd.DataFrame({'x1': x1, 'x2': x2, 'y': y})

## Simulation Loop


In [ ]:
results = []
correlations = [("No Corr", 0.0), ("Mild Corr", 0.5), ("High Corr", 0.99)]
models = {
    "Linear": LinearRegression(),
    "Ridge": Ridge(alpha=0.1),
    "Lasso": Lasso(alpha=0.1)
}

for label, rho in correlations:
    for i in range(10):
        df = generate_data(correlation=rho)
        X = df[['x1', 'x2']]
        y = df['y']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=i)

        for model_name, model in models.items():
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            results.append({
                'Model': model_name,
                'Correlation': label,
                'MSE': mse
            })

## Results Summary


In [ ]:
results_df = pd.DataFrame(results)
summary = results_df.groupby(['Model', 'Correlation']).agg(
    Mean_MSE=('MSE', 'mean'),
    SD_MSE=('MSE', 'std')
).reset_index()

summary

## Visualization (Optional)


In [ ]:
sns.boxplot(data=results_df, x='Correlation', y='MSE', hue='Model')
plt.title("Model MSE by Predictor Correlation")
plt.show()